<a href="https://colab.research.google.com/github/mysuikabar/colab_code_samples/blob/feature/langchain/output_fixing_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Output Parser
- Pydantic (JSON) parser
  - LLMからの出力をJSON形式で受け取る機能
  - https://python.langchain.com/docs/modules/model_io/output_parsers/pydantic
- Auto-fixing parser
  - 期待したJSON出力が得られない場合、自動でLLMに間違いを修正させる機能
  - https://python.langchain.com/docs/modules/model_io/output_parsers/output_fixing_parser

In [ ]:
!pip install python-dotenv openai langchain > /dev/null

In [ ]:
import os
from pathlib import Path
from pydantic import BaseModel, Field

import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
from langchain.prompts.chat import ChatPromptTemplate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ENV_PATH = "/content/drive/MyDrive/study/llm/.env"
load_dotenv(ENV_PATH)
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
# JSONオブジェクトのスキーマを定義
class Dish(BaseModel):
    name: str = Field(description="Name of dish.")
    ingredients: list[str] = Field(description="Ingredients needed to prepare the dish.")

# パーサーを定義
parser = PydanticOutputParser(pydantic_object=Dish)
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [ ]:
# プロンプトを作成
prompt = """List the ingredients needed to make an {dish}.
Answers must be given in Japanese.
{format_instruction}
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("human", prompt)]
)
message = prompt_template.format_messages(
    dish="麻婆豆腐",
    format_instruction=parser.get_format_instructions(),
)

In [ ]:
# LLMに投げる
chat = ChatOpenAI()
response = chat(message)

# 内容をパース
fixing_parser.parse(response.content)

Dish(name='麻婆豆腐', ingredients=['豆腐', '豚ひき肉', '豆板醤', '醤油', '砂糖', '酒', 'ラー油', 'ニンニク', 'しょうが', '水溶き片栗粉', '水', 'ごま油'])